In [1]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
from omegaconf import OmegaConf, open_dict
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

[NeMo W 2022-07-13 10:49:03 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [17]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-07-13 13:55:27 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-07-13 13:55:27 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-07-13 13:55:27 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-13 13:55:28 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-07-13 13:55:28 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-07-13 13:55:28 features:200] PADDING: 16
[NeMo I 2022-07-13 13:55:29 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [18]:
manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL1/manifest_S2002.json'
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
error = wer(transcript, predicted)
print('Word Error Rate (Resampled): ' + str(error))

Word Error Rate (Resampled): 0.3628964355879877


In [5]:
## Importing libraries 
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
import numpy as np
import soundfile as sf
from pathlib import Path
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel
import IPython.display as ipd
from pydub import AudioSegment

[NeMo W 2022-07-13 11:39:16 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [6]:
def infer(spec_gen_model, vocoder_model, str_input):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=None)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        mixing_enabled, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"S{new_speaker_id}_{mixing}"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [8]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_train.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S2002/train/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_train_manifestS2002.json'

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

n = 0
speakers = [2002]

with open(manifest_path, 'w') as fout: 
    for speaker_id in speakers: 
        mixing = False
        last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
        print(last_ckpt)
        spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
        spec_model.eval().cuda()

        with open(text_dir, "r") as f: 
            while True: 
              text = f.readline().strip()
              if text == '': 
                break 
              spec, audio = infer(spec_model, vocoder, text)
              audio = ipd.Audio(audio, rate=22050)
              audio = AudioSegment(audio.data)
              file_dir = audio_dir + str(n) + '.wav'
              audio.export(file_dir, format="wav")

              duration = librosa.core.get_duration(filename=file_dir)
              metadata = {
                "audio_filepath": file_dir,
                "duration": duration, 
                "text": text
              }
              json.dump(metadata, fout)
              fout.write('\n')   
              n = n + 1 

[NeMo I 2022-07-13 11:44:07 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-13 11:44:07 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-13 11:44:07 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-13 11:44:10 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-13 11:44:10 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-13 11:44:10 features:200] PADDING: 0


[NeMo W 2022-07-13 11:44:10 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-13 11:44:10 features:200] PADDING: 0
[NeMo I 2022-07-13 11:44:12 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
S2002_no_mixing/FastPitch/2022-07-07_10-47-10/checkpoints/FastPitch--v_loss=1.3819-epoch=24-last.ckpt
[NeMo I 2022-07-13 11:44:16 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-13 11:44:45 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-13 11:44:45 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL1/melSpect_S2002_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min

[NeMo I 2022-07-13 11:44:45 features:200] PADDING: 1


In [9]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_val.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S2002/val/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_val_manifestS2002.json'

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

n = 0
speakers = [2002]

with open(manifest_path, 'w') as fout: 
    for speaker_id in speakers: 
        mixing = False
        last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
        print(last_ckpt)
        spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
        spec_model.eval().cuda()

        with open(text_dir, "r") as f: 
            while True: 
              text = f.readline().strip()
              if text == '': 
                break 
              spec, audio = infer(spec_model, vocoder, text)
              audio = ipd.Audio(audio, rate=22050)
              audio = AudioSegment(audio.data)
              file_dir = audio_dir + str(n) + '.wav'
              audio.export(file_dir, format="wav")

              duration = librosa.core.get_duration(filename=file_dir)
              metadata = {
                "audio_filepath": file_dir,
                "duration": duration, 
                "text": text
              }
              json.dump(metadata, fout)
              fout.write('\n')   
              n = n + 1 

[NeMo I 2022-07-13 11:48:38 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-13 11:48:38 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-13 11:48:38 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-13 11:48:41 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-13 11:48:41 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-13 11:48:41 features:200] PADDING: 0


[NeMo W 2022-07-13 11:48:41 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-13 11:48:41 features:200] PADDING: 0
[NeMo I 2022-07-13 11:48:43 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
S2002_no_mixing/FastPitch/2022-07-07_10-47-10/checkpoints/FastPitch--v_loss=1.3819-epoch=24-last.ckpt
[NeMo I 2022-07-13 11:48:47 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-13 11:49:17 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-13 11:49:17 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL1/melSpect_S2002_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min

[NeMo I 2022-07-13 11:49:17 features:200] PADDING: 1


In [19]:
def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

quartznet.encoder.freeze()
quartznet.encoder.apply(enable_bn_se)
print("Model encoder has been frozen, and batch normalization has been unfrozen")

Model encoder has been frozen, and batch normalization has been unfrozen


In [20]:
quartznet.decoder.freeze()
quartznet.decoder.apply(enable_bn_se)
print("Model decoder has been frozen, and batch normalization has been unfrozen")

Model decoder has been frozen, and batch normalization has been unfrozen


In [21]:
## Config Information 
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
config_path = './configs/config.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
print(params)

{'name': 'QuartzNet15x5', 'sample_rate': 16000, 'repeat': 1, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'model': {'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}, 'preprocessor': {'_target_': 'nemo.collections

In [ ]:
train_manifest_path = '/home/dgxuser/Desktop/kailin/TTS_train_manifestS2002.json'
val_manifest_path = '/home/dgxuser/Desktop/kailin/TTS_val_manifestS2002.json'

# add in manifest paths 
params['model']['train_ds']['manifest_filepath'] = train_manifest_path
params['model']['validation_ds']['manifest_filepath'] = val_manifest_path

# Point to the new validation data for fine-tuning
quartznet.setup_validation_data(val_data_config=params['model']['validation_ds'])

# Point to the data we'll use for fine-tuning as the training set
quartznet.setup_training_data(train_data_config=params['model']['train_ds'])

# Create a PyTorch Lightning trainer and call `fit` again.
trainer = pl.Trainer(max_epochs=2, accelerator='gpu', devices=1)
trainer.fit(quartznet)

[NeMo I 2022-07-13 14:01:39 collections:193] Dataset loaded with 100 files totalling 0.12 hours
[NeMo I 2022-07-13 14:01:39 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2022-07-13 14:01:39 collections:193] Dataset loaded with 2983 files totalling 4.43 hours
[NeMo I 2022-07-13 14:01:39 collections:194] 0 files were filtered totalling 0.00 hours


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
[NeMo W 2022-07-13 14:01:39 modelPT:484] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-07-13 14:01:39 modelPT:585] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )


[NeMo W 2022-07-13 14:01:39 lr_scheduler:816] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 29.7 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
77.3 K    Trainable params
18.8 M    Non-trainable params
18.9 M    Total params
75.698    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [15]:
manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL1/manifest_S2002.json'
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
error = wer(transcript, predicted)
print('Word Error Rate (Resampled): ' + str(error))

Word Error Rate (Resampled): 0.3874543923659837
